In [4]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
idx = pd.IndexSlice

In [9]:
with open('data/tyres', 'rb') as file:
    mtyres = pickle.load(file)
with open('data/fuel', 'rb') as file:
    mfuel = pickle.load(file)
mtyres

{'Extra Soft': 0.01016453846642526,
 'Hard': 0.005913494030186977,
 'Medium': 0.006243099132201616,
 'Rain': 0.005171909315288629,
 'Soft': 0.010270779055422377}